# Abril

---

In [57]:
import numpy as np
from datetime import date, timedelta
import string
import time
from pandas import DataFrame
from os import getcwd
from pygsheets import authorize
import pivottablejs

## Integração das APIs do GCP - das contas google 

### Conta RIO

In [2]:
credenciaisRio = authorize(service_file=getcwd()+'/keyapirio.json')

#### na célula abaixo os Links estarão ocultos

In [8]:
linkAgendamentoRio = 'basta copiar da pagina do google sheet'
abaAgendamentoRio = 'xxxxxxxx'

#### Código de integração da APIRest google sheets

In [3]:
# acessando planilha
#planilhaAgendamentoRio = credenciaisRio.open_by_key(linkAgendamentoRio)
#tabelaAgendamentoRio = planilhaAgendamentoRio.spreadsheet_ids(abaAgendamentoRio)

### Conta RIO2

### Conta Carioca

Nessa etapa estamos acessando a base de dados do google via API, que em outras palavras se remete a integração de sistemas externos

In [58]:
credenciaisCarioca = authorize(service_file=getcwd()+'/keysurveycarioca.json')

In [59]:
link = 'https://docs.google.com/spreadsheets/d/essa parte é a informação que identifica a planilha que se deseja integrar/'

In [60]:
# acessando planilha
planilha = credenciaisCarioca.open_by_url(link)
tabela = planilha.worksheet_by_title("Respostas ao formulário 1")

### Quadro de Dados (DataFrame)

Nessa etapa estamos transformando a base de atualizada até o momento da última integração realizada acima. Em um quadro de dados, para que assim possamos extrair a riqueza que a base tem a nos oferecer.

In [81]:
import datetime

Nessa parte estamos criando o objeto com todos os dados, que até a data informada

In [88]:
df_carioca_agend = DataFrame(tabela)
print(f'Na data {datetime.date.today().isoformat()} esse quadro de dados está sendo criado.')

Na data 2024-05-03 esse quadro de dados está sendo criado.


In [90]:
print(f'Até o momento informado na linha de cima, o agendamento foi aplicado {df_carioca_agend.shape[0] - 1} pessoas.')

Até o momento informado na linha de cima, o agendamento foi aplicado 1519 pessoas.


In [91]:
df_carioca_agend.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,Carimbo de data/hora,LOCAL,NOME COMPLETO,,BAIRRO,CEP,DATA DE NASCIMENTO,TELEFONE,EMAIL,CPF,...,ENDEREÇO COMPLETO,CPF,Como soube do Projeto de Castração de cães e g...,,,,,,,


In [92]:
df_carioca_agend = df_carioca_agend[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]]
df_carioca_agend.columns = list(df_carioca_agend[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]].iloc[0])

Nessa etapa vamos informar os atributos ou variáveis que tem o quadro de dados, que em outras palavras são as perguntas (que se transformam em colunas na base de dados) respondidas no formulário __agendamento__ 

In [104]:
print (f'Segue a lista com todas variáveis da base de dados: {list(df_carioca_agend.columns)}')
print (f'''\nVamos utilizar as variáveis: {list(df_carioca_agend.columns)[0]}, {list(df_carioca_agend.columns)[11]}, 
{list(df_carioca_agend.columns)[12]} e {list(df_carioca_agend.columns)[13]}.''')

Segue a lista com todas variáveis da base de dados: ['Carimbo de data/hora', 'LOCAL', 'NOME COMPLETO', '', 'BAIRRO', 'CEP', 'DATA DE NASCIMENTO', 'TELEFONE', 'EMAIL', 'CPF', 'NOME DO ANIMAL', 'ESPÉCIE', 'GÊNERO', 'STATUS/ATENDIMENTO', 'MOTIVO', 'ENDEREÇO COMPLETO', 'CPF', 'Como soube do Projeto de Castração de cães e gatos?']

Vamos utilizar as variáveis: Carimbo de data/hora, ESPÉCIE, 
GÊNERO e STATUS/ATENDIMENTO.


In [66]:
df_carioca_agend = df_carioca_agend.drop(0, axis=0)

In [67]:
df_carioca_agend.head(1)

,Carimbo de data/hora,LOCAL,NOME COMPLETO,,BAIRRO,CEP,DATA DE NASCIMENTO,TELEFONE,EMAIL,CPF,NOME DO ANIMAL,ESPÉCIE,GÊNERO,STATUS/ATENDIMENTO,MOTIVO,ENDEREÇO COMPLETO,CPF,Como soube do Projeto de Castração de cães e gatos?
1,25/02/2024 10:29:53,Adelazil Policardio de Oliveira,Av. Cidade de Deus 325 casa 3,,Cidade de Deus,22773-200,28/05/1958,979467197,,Gigi,21616450487,CANINO,FEMEA,,,,,


In [77]:
relatorio_agendamento_carioca = DataFrame(DataFrame(df_carioca_agend['Carimbo de data/hora'].str.split(' ', expand=True)[0]).join(df_carioca_agend[['ESPÉCIE','GÊNERO','STATUS/ATENDIMENTO']]).groupby(by=['STATUS/ATENDIMENTO']).value_counts()).reset_index()

In [79]:
relatorio_agendamento_carioca.loc[relatorio_agendamento_carioca['STATUS/ATENDIMENTO']== 'CIRURGIA REALIZADA']

,STATUS/ATENDIMENTO,0,ESPÉCIE,GÊNERO,count
0,CIRURGIA REALIZADA,,FELINO,FEMEA,1
4,CIRURGIA REALIZADA,01/04/2024,FELINO,MACHO,8
5,CIRURGIA REALIZADA,01/04/2024,CANINO,MACHO,7
6,CIRURGIA REALIZADA,01/04/2024,FELINO,FEMEA,5
7,CIRURGIA REALIZADA,01/04/2024,CANINO,FEMEA,5
...,...,...,...,...,...
358,CIRURGIA REALIZADA,30/04/2024,FELINO,MACHO,1
359,CIRURGIA REALIZADA,31/03/2024,FELINO,FEMEA,9
360,CIRURGIA REALIZADA,31/03/2024,CANINO,MACHO,2
361,CIRURGIA REALIZADA,31/03/2024,FELINO,MACHO,2


In [78]:
relatorio_agendamento_carioca

,STATUS/ATENDIMENTO,0,ESPÉCIE,GÊNERO,count
0,CIRURGIA REALIZADA,,FELINO,FEMEA,1
1,RECUSA,01/03/2024,CANINO,MACHO,1
2,CIRURGIA REALIZADA,01/03/2024,FELINO,FEMEA,1
3,CIRURGIA REALIZADA,01/03/2024,CANINO,MACHO,1
4,CIRURGIA REALIZADA,01/04/2024,FELINO,MACHO,8
...,...,...,...,...,...
358,CIRURGIA REALIZADA,30/04/2024,FELINO,MACHO,1
359,CIRURGIA REALIZADA,31/03/2024,FELINO,FEMEA,9
360,CIRURGIA REALIZADA,31/03/2024,CANINO,MACHO,2
361,CIRURGIA REALIZADA,31/03/2024,FELINO,MACHO,2


In [56]:
import numpy as np
# natsort import index_natsorted
#DataFrame(teste).reset_index().sort_values(by=[0], key=lambda x=np.argsort(ind) )

---